In [1]:
#### load packages, *.h5 folder and exp.h5 file location
%matplotlib widget

from essential_func import *
from analysis_data import *

qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

default_sample_dir = 'July-2021-Sample#6'
csv_and_code_abs_directory = '/Users/bashit.a/Documents/Alzheimer/Codes/'
samples_csv = 'data_directory.csv'

dropdown_name_list, cwd, exp_folder = change_python_path(dropdown_name = default_sample_dir, csv_file_location = csv_and_code_abs_directory, samples_csv = samples_csv)

In [103]:
### background subtraction
%matplotlib widget

file = '2029_Dentate_gyrus-roi0_masked.h5'
QSearchStart, QSearchEnd = 1.2, 1.3   # scaling regions 295,310 or may be 290,370
frame = 981                          # amyloid frame    2428, 3101, 3923, 2190,  691, 1134, 458
tissue_frame = 1480                     # tissue frame
bkg_frame = 338                      # background frame
seek_mf = (-8,8,0.01)        #gray        # mf goes from -8 to +8 by 0.01
window_size=4                         # window size for moving average filter

### Tissue scaled background subtraction
data = Data_Analysis(file, qgrid2)
Iq = data.bkg_sub(bkg_frame = bkg_frame)
mf, area = data.scaling_frame(frame, tissue_frame, mf_Qindices = (QSearchStart, QSearchEnd), seek_mf = seek_mf, window_size=window_size, show_result=True)
data.plot()

MF =  3.02 AREA: 1.3~1.42 =  -2.6661509785996312e-05 MSE =  1.416545839814888e-05


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/bashit.a/Documents/Alzheimer/Codes/analysis_data.py:231: RuntimeWarning: divide by zero encountered in log
  axs[0,1].plot(self.qgrid, np.log(self.IqBS[self.input_fr]))


In [18]:
### Load all frames mf, and area data

sc_data = {}; sc_data['mf'] = np.zeros(data.n_patterns); sc_data['area'] = np.zeros(data.n_patterns)
for frame in range(data.n_patterns):
    sc_data['mf'][frame] , sc_data['area'][frame]   =  data.scaling_frame(frame, tissue_frame, mf_Qindices = (QSearchStart, QSearchEnd), seek_mf = seek_mf, window_size=window_size, show_result=False)

In [19]:
### Draw heat map depending on background subtracted descending area 
%matplotlib widget

file = file
sort_by = sc_data['area']


sorted_idx  = np.argsort(sort_by, kind='stable')[::-1][:50] # sc_data['area'].shape = (4941,)  # first 50 area sorted descending

diff_patterns = np.zeros(len(sort_by))
diff_patterns[sorted_idx] = sort_by[sorted_idx]
Width, Height = width_height(file)
img_orig = snaking(Width, Height, diff_patterns)

f,ax = plt.subplots()
plot_heat_map_from_data(img_orig, Width, Height, args = (f, ax), title= f'Heat Map {file}')
[print('Fr-',idx, 'value: ' , np.round(diff_patterns[idx],8)) for idx in sorted_idx]
pass

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fr- 2080 value:  0.00038841
Fr- 2079 value:  0.00031501
Fr- 2067 value:  0.00029591
Fr- 2081 value:  0.00026183
Fr- 2068 value:  0.00024662
Fr- 2066 value:  0.00024051
Fr- 1958 value:  0.00023657
Fr- 4620 value:  0.00017625
Fr- 1957 value:  0.00017373
Fr- 4621 value:  0.00016942
Fr- 4529 value:  0.00016058
Fr- 4528 value:  0.00015589
Fr- 4622 value:  0.00015274
Fr- 2078 value:  0.00011512
Fr- 1959 value:  0.00011449
Fr- 458 value:  8.982e-05
Fr- 4530 value:  8.335e-05
Fr- 4619 value:  7.551e-05
Fr- 2189 value:  7.194e-05
Fr- 4527 value:  6.896e-05
Fr- 1956 value:  6.554e-05
Fr- 459 value:  5.998e-05
Fr- 2190 value:  5.977e-05
Fr- 4623 value:  5.807e-05
Fr- 1698 value:  5.638e-05
Fr- 4625 value:  5.455e-05
Fr- 1841 value:  5.436e-05
Fr- 2429 value:  5.345e-05
Fr- 750 value:  5.329e-05
Fr- 3802 value:  5.313e-05
Fr- 3123 value:  5.312e-05
Fr- 457 value:  5.248e-05
Fr- 4808 value:  5.241e-05
Fr- 4501 value:  5.183e-05
Fr- 2428 value:  5.151e-05
Fr- 2069 value:  5.15e-05
Fr- 1475 value:  5

In [102]:
### Produce heatmap and cluster frames by thresholding
%matplotlib widget
from scipy.ndimage import label

# spec
file = '2029_Dentate_gyrus-roi0_masked.h5'
scattering = '_WAXS2'
scattering_max = 0.0361
thr            = 0.034
scattering_inc = 0.001


# computation
Width, Height = width_height(file)
img_orig =  discritize_scattering(file, qgrid2, scattering, data_binning=True, bins=np.fromiter(drange(0, scattering_max, scattering_inc), float) )
gray_img,_ = global_thresholding(img_orig, thr, binary_inv = False)

# plot
f,axs = plt.subplots(nrows=1, ncols=2)
plot_heat_map_from_data(img_orig, Width, Height, args = (f, axs[0]), title= f'Heat Map {scattering}', cmap="jet")
plot_heat_map_from_data(gray_img, Width, Height, args = (f, axs[1]), title= f'Heat Map Thr at {thr}', cmap="jet")
plt.suptitle(f'{file}')
plt.tight_layout()

labeled_array, num_features = label(gray_img, np.ones((3,3)))
print('Total clusters = ', num_features)

Width, Height = labeled_array.shape[1], labeled_array.shape[0]
sna = snaking(Width, Height , np.arange(0, Width*Height))


dict_label = {}
for i in np.unique(labeled_array):
    if i!=0:
        dict_label[i] = sna[labeled_array==i]
        print(f'label {i}: {dict_label[i]}')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Total clusters =  40
label 1: [7807 7690 7680 7681 7565]
label 2: [7556 7557 7438 7437 7436 7435 7429 7430 7431 7432 7311 7310 7309 7308
 7306]
label 3: [7048 7049 6937 6936 6935 6922 6923]
label 4: [7055 6930]
label 5: [6798 6799 6800 6684 6683 6682 6681 6671 6672 6673 6674 6558 6557 6556
 6555 6546 6547 6548 6431 6430 6429 6420 6421 6422 6304 6303]
label 6: [6174 6173 6048 6047]
label 7: [5931 5930 5912 5913 5806 5805 5785 5680 5679 5659 5660 5554 5553 5552
 5533 5534 5535 5428 5427 5426 5408 5409 5301]
label 8: [5802]
label 9: [5058 5057]
label 10: [4798 4797 4778]
label 11: [4677]
label 12: [4415 4404]
label 13: [4278 4279 4280 4163 4162 4161 4152 4153 4154 4037 4036]
label 14: [4149 4040]
label 15: [4156]
label 16: [3911 3910 3900 3901 3785 3784 3783 3782]
label 17: [3749 3750 3685 3684 3683]
label 18: [3639 3640 3542]
label 19: [3287 3286]
label 20: [3260]
label 21: [3271 3154 3153 3145 3146]
label 22: [2890]
label 23: [2800 2799 2798 2743 2744 2745 2675 2674]
label 24: [2507 240

In [52]:
np.sort(dict_label[35])

array([ 907,  908,  980,  981,  982, 1034, 1035, 1107])

In [12]:
#### check generated heat maps collected frame data
%matplotlib widget
def exec_dropdown_value(frame):
    
    mf,area =  data.scaling_frame(frame, tissue_frame, mf_Qindices = (QSearchStart, QSearchEnd), seek_mf = seek_mf, window_size=window_size, show_result=True)
    data.plot()
    
frame = ipywidgets.Dropdown(options= sorted_area_idx,  description='Frame:',    disabled=False, continuous_update=False)
ipywidgets.interact(exec_dropdown_value, frame=frame)
print(sorted_area_idx)

interactive(children=(Dropdown(description='Frame:', options=(2080, 2079, 2067, 2081, 2068, 2066, 1958, 4620, …

[2080 2079 2067 2081 2068 2066 1958 4620 1957 4621 4529 4528 4622 2078
 1959  458 4530 4619 2189 4527 1956  459 2190 4623 1698 4625 1841 2429
  750 3802 3123  457 4808 4501 2428 2069 1475 2188 2451  520 1477 4135
 4526  894 1476 3660 4498 2183 4499 4931]


In [21]:
#### generate Radius of gyration
%matplotlib widget

from tqdm import tqdm

# specs
file = '2048_B8_masked.h5'
Nsplits  = 2                                        # number of lines to fit
StartIdx = 4                                        # including start index
LastIdx  = 52                                       # Excluding last index
frame    = 2223
max_frames = 40                                    # first 100 frame values
Rxc_str = ['Rxc-1', 'Rxc-2',]

# new part
data = Data_Analysis(file, qgrid2)
IqBS = data.bkg_sub(bkg_frame = 2223)

df_IqBS = pd.DataFrame(IqBS)    # convert to dataframe
list_max_frames = df_IqBS.sort_values(5,ascending=False)[:max_frames].index.to_list()   # 5-->th column values sorted in descending order and taking max 40 frames intensity to fit lines except are zero
print(list_max_frames)
Rxc = {}
n_patterns = len(IqBS)

for rxc_str in Rxc_str:
    Rxc[rxc_str] = np.zeros(n_patterns)     # except list_max_frames all values are zero

    for frame in tqdm(list_max_frames):
        df = optimize_best_lines(IqBS[frame], qgrid2, Nsplits, LastIdx, StartIdx,)   # get dataframe for the optimized one
        try:
            Rxc[rxc_str][frame] = df.iloc[df['rsq'].idxmax()][rxc_str]    
        except:
            print(f'Frame = {frame} {rxc_str} optimization failed - setting it to zero')
            Rxc[rxc_str][frame] = 0                 # set to zero in nan is found in try section
        
# plot heat maps
Width, Height = width_height(file)
f, axs = plt.subplots(1, len(Rxc_str), num=f'{file} Heat maps', figsize=(10,5))
for i, rxc_str in enumerate(Rxc_str):
    diff_patterns = Rxc[rxc_str]
    img_orig = snaking(Width, Height, diff_patterns)
    
    plot_heat_map_from_data(img_orig, Width, Height, args = (f, axs[i]), title= f'{file}- {rxc_str}')

  0%|          | 0/40 [00:00<?, ?it/s]/Users/bashit.a/Documents/Alzheimer/Codes/essential_func.py:832: RuntimeWarning: divide by zero encountered in log
  IqQ = np.log(IqBS*qgrid)                            # background subtracted Iq
  5%|▌         | 2/40 [00:00<00:02, 17.10it/s]

[2067, 1958, 1326, 2080, 1358, 2428, 2068, 4374, 2197, 2199, 4373, 1357, 2427, 2079, 1957, 2452, 2198, 1327, 4912, 2200, 1325, 4223, 2558, 4913, 4723, 2066, 4848, 4911, 1359, 4847, 2437, 2081, 2323, 4222, 2322, 2196, 1959, 2320, 2438, 2453]


 12%|█▎        | 5/40 [00:00<00:01, 19.54it/s]/Users/bashit.a/Documents/Alzheimer/Codes/essential_func.py:832: RuntimeWarning: invalid value encountered in log
  IqQ = np.log(IqBS*qgrid)                            # background subtracted Iq
/Users/bashit.a/Documents/Alzheimer/Codes/essential_func.py:859: RuntimeWarning: invalid value encountered in sqrt
  df.loc[idx,f'Rxc-{i+1}'] = np.round(np.sqrt(-df.loc[idx,f'slope-{i+1}']*2),4)
100%|██████████| 40/40 [00:01<00:00, 22.18it/s]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
### get individual frame fitting of radius of gyration
%matplotlib widget
# specs
frame = 2067
# verification
df = optimize_best_lines(IqBS[frame], qgrid2, Nsplits, LastIdx, StartIdx, print_summary=True, show_plot=True)
print(df)

Total Combinations = 45

Summary of results Rsq -- 
 0               4.0000
1              18.0000
2              18.0000
3              52.0000
Tpoints-1      14.0000
qgridL-1        0.0090
qgridH-1        0.0230
slope-1     -4833.9374
Rxc-1          98.3254
I(0)-1          0.9474
rsq-1           0.9472
std_err-1     329.6247
Tpoints-2      34.0000
qgridL-2        0.0230
qgridH-2        0.0640
slope-2      -698.0548
Rxc-2          37.3646
I(0)-2         -1.2604
rsq-2           0.9421
std_err-2      30.5975
rsq             1.8893
Name: 12, dtype: float64
More Summary of results Rsq -- 
     0   1   2   3  Tpoints-1  qgridL-1  qgridH-1     slope-1     Rxc-1  \
0   4   6   6  52          2     0.009     0.011 -12982.1460  161.1344   
1   4   7   7  52          3     0.009     0.012 -11530.7305  151.8600   
2   4   8   8  52          4     0.009     0.013 -10360.3461  143.9468   
3   4   9   9  52          5     0.009     0.014  -9429.4692  137.3279   
4   4  10  10  52          6     0.0

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

    0   1   2   3  Tpoints-1  qgridL-1  qgridH-1     slope-1     Rxc-1  \
0   4   6   6  52          2     0.009     0.011 -12982.1460  161.1344   
1   4   7   7  52          3     0.009     0.012 -11530.7305  151.8600   
2   4   8   8  52          4     0.009     0.013 -10360.3461  143.9468   
3   4   9   9  52          5     0.009     0.014  -9429.4692  137.3279   
4   4  10  10  52          6     0.009     0.015  -8636.6298  131.4278   
5   4  11  11  52          7     0.009     0.016  -7878.8447  125.5296   
6   4  12  12  52          8     0.009     0.017  -7300.9062  120.8380   
7   4  13  13  52          9     0.009     0.018  -6807.7529  116.6855   
8   4  14  14  52         10     0.009     0.019  -6297.4656  112.2271   
9   4  15  15  52         11     0.009     0.020  -5884.7404  108.4872   
10  4  16  16  52         12     0.009     0.021  -5497.3718  104.8558   
11  4  17  17  52         13     0.009     0.022  -5129.8891  101.2906   
12  4  18  18  52         14     0.009

In [15]:
pd.set_option('display.max_columns',None)
df.style.background_gradient()
df[df['rsq'] > 1.86]                      # Nsplits
#df[df['rsq-3'] > 0.99]['Tpoints-3'].idxmax()      # Nsplits
#plot_linear_lines(xData, yData, best_indices)

,0,1,2,3,Tpoints-1,qgridL-1,qgridH-1,slope-1,Rxc-1,I(0)-1,rsq-1,std_err-1,Tpoints-2,qgridL-2,qgridH-2,slope-2,Rxc-2,I(0)-2,rsq-2,std_err-2,rsq
2,4,8,8,52,4,0.009,0.013,-10360.3461,143.9468,1.6805,0.9908,707.4576,44,0.013,0.064,-921.5366,42.9310,-0.7508,0.8726,54.3384,1.8634
3,4,9,9,52,5,0.009,0.014,-9429.4692,137.3279,1.5856,0.9869,626.9592,43,0.014,0.064,-893.2780,42.2677,-0.8131,0.8823,50.9652,1.8692
4,4,10,10,52,6,0.009,0.015,-8636.6298,131.4278,1.4986,0.9828,570.7240,42,0.015,0.064,-866.6187,41.6322,-0.8723,0.8912,47.8814,1.8740
5,4,11,11,52,7,0.009,0.016,-7878.8447,125.5296,1.4093,0.9761,551.1112,41,0.016,0.064,-840.9667,41.0114,-0.9297,0.9000,44.8855,1.8761
6,4,12,12,52,8,0.009,0.017,-7300.9062,120.8380,1.3361,0.9733,493.7115,40,0.017,0.064,-817.0807,40.4248,-0.9836,0.9078,42.2534,1.8811
7,4,13,13,52,9,0.009,0.018,-6807.7529,116.6855,1.2693,0.9709,445.6174,39,0.018,0.064,-794.8459,39.8709,-1.0342,0.9144,39.9717,1.8853
8,4,14,14,52,10,0.009,0.019,-6297.4656,112.2271,1.1953,0.9642,429.2031,38,0.019,0.064,-772.9985,39.3192,-1.0843,0.9214,37.6173,1.8856
9,4,15,15,52,11,0.009,0.020,-5884.7404,108.4872,1.1313,0.9607,396.8797,37,0.020,0.064,-752.8977,38.8046,-1.1309,0.9272,35.6586,1.8879
10,4,16,16,52,12,0.009,0.021,-5497.3718,104.8558,1.0673,0.9559,373.3956,36,0.021,0.064,-733.6226,38.3046,-1.1760,0.9327,33.8078,1.8886
11,4,17,17,52,13,0.009,0.022,-5129.8891,101.2906,1.0027,0.9497,355.9882,35,0.022,0.064,-714.7376,37.8084,-1.2206,0.9383,31.9132,1.8880


In [16]:
%matplotlib widget

# specs
frame = 617    # 2048_B8_masked frame = 617 Rxc_2 = nan
QQ  = np.square(qgrid2)                            # squaring q values
IqQ = np.log(IqBS[frame]*qgrid2)                         # background subtracted Iq

# semi specs
xData = QQ
yData = IqQ

# computation
def update_plot(Q1, Q2H):
    Q1L, Q1H= Q1
    plot_linear_lines(xData=xData, yData=yData, indices=[Q1L,Q1H,Q1H,Q2H])

Q1 =  ipywidgets.IntRangeSlider(min=0, max=109, value = [2, 50], step=1)
Q2H = ipywidgets.IntSlider(min=0, max=109, value = 109, step=1)

ipywidgets.interact(update_plot, Q1=Q1, Q2H=Q2H)

<ipython-input-16-1aa1b51b47a5>:6: RuntimeWarning: divide by zero encountered in log
  IqQ = np.log(IqBS[frame]*qgrid2)                         # background subtracted Iq
<ipython-input-16-1aa1b51b47a5>:6: RuntimeWarning: invalid value encountered in log
  IqQ = np.log(IqBS[frame]*qgrid2)                         # background subtracted Iq


interactive(children=(IntRangeSlider(value=(2, 50), description='Q1', max=109), IntSlider(value=109, descripti…

<function __main__.update_plot(Q1, Q2H)>

In [99]:
# 1-d plot for high intensities
%matplotlib widget
import mplcursors
from matplotlib.widgets import CheckButtons

# spec
frames   = np.sort( np.unique( np.concatenate([np.array([1480]), dict_label[24], dict_label[32], dict_label[35], dict_label[36] ]) ) ) # np.array(sorted(list_max_frames))
print(frames)
# background subtraction
file = '2029_Dentate_gyrus-roi0_masked.h5'      ##### change here
data = Data_Analysis(file, qgrid2)
IqBS = data.bkg_sub(bkg_frame = 1480)           ##### change here


# plot
f, ax = plt.subplots(2,2, figsize = (16,8), num=f'data', constrained_layout = False) 
plt.subplots_adjust(left=0.15)

saxs_plots = []    
waxs_plots = []
Iq = read_Iq(file, scattering='merged')

for frame in frames:  
    QQ  = qgrid2                               ##### change here                # squaring q values
#     IqQ = (Iq[frame])                        ##### change here Iq

#     QQ  = np.square(qgrid2)                    ##### change here                # squaring q values
    IqQ = (IqBS[frame]*qgrid2)                 ##### change here # background subtracted IqBS
#     IqQ = IqBS[frame]                 ##### change here # background subtracted IqBS

    # semi specs
    xData = QQ
    yData = IqQ
    
    saxs_plots.append(ax[0,0].plot(xData[130:], yData[130:], visible=False, label = f'{frame}' ))   # here dataset has only one frame
    ax[0,0].set_xlabel("$q (\AA^{-1})$",); 
    ax[0,0].set_ylabel("$I$",);
    ax[0,0].set_xscale('linear')
    ax[0,0].set_yscale('linear')

    ax[1,0].set_xlabel("$q (\AA^{-1})$",); 
    ax[1,0].set_ylabel("$I$",);
    ax[1,0].set_xscale('linear')
    ax[1,0].set_yscale('log')
    waxs_plots.append(ax[1,0].plot(xData[2:50] , yData[2:50], visible=False,label=f'{frame}')); 
    
# plot heatmap
plot_heat_map_from_data(img_orig, Width, Height, args = (f, ax[0,1]), cmap='jet')
#plt.tight_layout()

# Make checkbuttons with all plotted lines with correct visibility
rax = plt.axes([0.005, 0.1, 0.05, 0.6])
labels = [str(line[0].get_label()) for line in saxs_plots]   #label is a string
visibility = [line[0].get_visible() for line in saxs_plots]
check = CheckButtons(rax, labels, visibility, )
for r,l in zip(check.rectangles, check.lines):
    r.set_width(0.1)
    r.set_alpha(0.4)
    for ll in l:
        ll.set_linewidth(8)
    
# [ll.set_linewidth(8) for l in check.lines for ll in l]
def func(label):
    index = labels.index(label)
    saxs_plots[index][0].set_visible(not saxs_plots[index][0].get_visible())
    waxs_plots[index][0].set_visible(not waxs_plots[index][0].get_visible())
    
    ax[0,0].legend(loc='upper right', bbox_to_anchor=(1, 1), fontsize='xx-small')
    ax[1,0].legend(loc='upper right', bbox_to_anchor=(1, 1), fontsize='xx-small')

    plt.draw()

check.on_clicked(func)
plt.subplot_tool()
#mplcursors.cursor()

[ 907  908  980  981  982 1034 1035 1107 1258 1260 1288 1289 1355 1356
 1357 1385 1414 1415 1416 1480 1481 1482 2278 2279 2280 2380 2381 2382
 2383 2405 2406 2507]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
import mplcursors
from matplotlib.widgets import CheckButtons

class Check_Box_Plots():
    
    def __init__(self, X, Y, labels, ax, ): 
        
        # initialize variable
        self.plots = []
        self.X = X
        self.Y = Y
        self.labels = labels
        self.ax = ax
    
    def plot_data(self, xlabel="$q(\AA^{-1})$", ylabel="$I$", xscale='linear', yscale='linear'):
        # plots checkbox data without visibility
        for idx in range(len(self.Y)):
            self.plots.append(self.ax.plot(self.X, self.Y[idx], visible=True, label = f'{self.labels[idx]}' ))   # here dataset has only one frame
            self.ax.set(xlabel = xlabel, ylabel=ylabel, xscale=xscale, yscale=yscale)

    # call when checkbox items clicked
    def func(self,label):
        """
            only global variable is plots, labels, ax
        """
        index = self.labels.index(label)

        self.plots[index][0].set_visible(not self.plots[index][0].get_visible())                # visible plot when clicked
        self.ax.legend(loc='upper right', bbox_to_anchor=(1, 1), fontsize='xx-small')      # visible lengend when clicked
        #plt.draw()

# instantiate object
# inputs data
X, Y =  np.square(qgrid2)    , IqBS[[2080, 1956]]*qgrid2 
labels = ['2080', '1956']
f,ax = plt.subplots(2,2,constrained_layout = False) 
plots = Check_Box_Plots(X, Y, labels, ax[1,1])
plots.plot_data()

# Make checkbuttons and attributes with all plotted lines with correct visibility
rax        = plt.axes([0.005, 0.1, 0.05, 0.6])              # checkbox axis location
labels     = [str(line[0].get_label()) for line in plots.plots]   # label is a string
visibility = [line[0].get_visible()    for line in plots.plots]   # visibility
check      = CheckButtons(rax, labels, visibility, )
for r,l in zip(check.rectangles, check.lines):
    r.set_width(0.1)                # checkbox width
    r.set_alpha(0.4)                
    for ll in l:
        ll.set_linewidth(8)         # check/uncheck linewidth or [ll.set_linewidth(8) for l in check.lines for ll in l]    

check.on_clicked(plots.func)              # check.get_status()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0

In [22]:
## Do cluster labeling
import cv2
from scipy.ndimage import label

img_gray = np.asarray(img_orig, dtype=np.uint8)
_, thresh_img = cv2.threshold(img_gray, 0, 255,cv2.THRESH_BINARY); #plt.imshow(thresh_img, cmap='gray')
labeled_array, num_features = label(thresh_img, np.ones((3,3)))
print('Total clusters = ', num_features)

Width, Height = labeled_array.shape[1], labeled_array.shape[0]
sna = snaking(Width, Height , np.arange(0, Width*Height))

[print('label = ', i, sna[labeled_array==i]) for i in np.unique(labeled_array) if i!=0]

Total clusters =  11
label =  1 [4911 4912 4913 4848 4847]
label =  2 [4723]
label =  3 [4374 4373]
label =  4 [4223 4222]
label =  5 [2558]
label =  6 [2452 2453 2428 2427]
label =  7 [2438 2437 2320]
label =  8 [2322 2323]
label =  9 [2196 2197 2198 2199 2200]
label =  10 [2079 2080 2081 2068 2067 2066 1957 1958 1959]
label =  11 [1357 1358 1359 1327 1326 1325]


[None, None, None, None, None, None, None, None, None, None, None]

In [21]:
from generate_files import *
save_mat('IqBS', ('IqBS', 'qgrid2'), (IqBS, qgrid2))

In [22]:
### Correlation coefficients analysis

Width, Height = 61, 81
sna = snaking(Width, Height , np.arange(0, Width*Height))
print(sna)

grid = np.zeros((Height,Width),dtype=object)
for i in range(0,Height):
    for j in range(0,Width):
        grid[i,j]=(i,j)
grid = grid[::-1]
print(grid)

[[4880 4881 4882 ... 4938 4939 4940]
 [4879 4878 4877 ... 4821 4820 4819]
 [4758 4759 4760 ... 4816 4817 4818]
 ...
 [ 122  123  124 ...  180  181  182]
 [ 121  120  119 ...   63   62   61]
 [   0    1    2 ...   58   59   60]]
[[(80, 0) (80, 1) (80, 2) ... (80, 58) (80, 59) (80, 60)]
 [(79, 0) (79, 1) (79, 2) ... (79, 58) (79, 59) (79, 60)]
 [(78, 0) (78, 1) (78, 2) ... (78, 58) (78, 59) (78, 60)]
 ...
 [(2, 0) (2, 1) (2, 2) ... (2, 58) (2, 59) (2, 60)]
 [(1, 0) (1, 1) (1, 2) ... (1, 58) (1, 59) (1, 60)]
 [(0, 0) (0, 1) (0, 2) ... (0, 58) (0, 59) (0, 60)]]


In [23]:
frame = 4940
def frame_to_idx(frame):
    idx = [i for i in grid[sna==frame]]
    return idx

def idx_to_frame(l):
    idx = []
    for i in l:
        a, b = i
        x = [i==(a,b) for i in grid[Height-a-1]]
        idx.append(sna[Height-a-1, np.where(x)].tolist())
    return flatten(idx)

l = frame_to_idx(frame)
print(frame_to_idx(frame))
print(idx_to_frame(l))

[(80, 60)]
[4940]


In [24]:
### locating valid neighbor indices 3*3 kernel
a, b = 80, 60
l = []
for i in range(a-1,a+2):
    for j in range(b-1,b+2):
        if (i,j) !=(a,b) and i>=0 and j>=0 and i<Height and j <Width:
            l.append((i,j))
print(l)

[(79, 59), (79, 60), (80, 59)]


In [25]:
## look for frames gotten from previous indices
idx = []
for i in l:
    a, b = i
    x = [i==(a,b) for i in grid[Height-a-1]]
    idx.append(sna[Height-a-1, np.where(x)].tolist())
flatten(idx)

[4820, 4819, 4939]

In [26]:
from scipy.io import loadmat
annots = loadmat('2048_B8_Iq.mat')

In [27]:
### Correlation coefficient calculation on individual frame
%matplotlib widget
from functools import reduce

def coeffs_map(file, ref_frame, thr_coeff, search_q, normalize, L):
    b = np.ones((1,L))/L    # numerator co-effs of filter transfer function
    a = np.ones(1)          # denominator co-effs of filter transfer function

    q_min, q_max = search_q
    Width, Height = width_height(file)
    Iq = read_Iq(file, 'merged')

    Iq = signal.fftconvolve(Iq,b,mode='same',) if L>1 else Iq         # filter output using convolution
    Iq = Iq/np.max(Iq,axis=1).reshape(-1,1) if normalize else Iq      # test how normalizing affecing data points print(Iq[0][:10])

    q_min_idx, q_max_idx = qgrid_to_indices(qgrid2, q_min), qgrid_to_indices(qgrid2, q_max)
    sna = snaking(Width, Height)

    cluster = []; coeffs = []
    for frame in range(Width * Height):
        coeff = np.corrcoef(Iq[ref_frame][q_min_idx:q_max_idx], Iq[frame][q_min_idx:q_max_idx])[0,1]
        if coeff >= thr_coeff:
            cluster.append(frame)
            coeffs.append(coeff)

    A = np.array([np.zeros((Height,Width)),sna])   # zero values matrix (A[0]=0) with frame numbers depth (A[1]=frames)

    A = from_clusterFr_ceffs_to_matrix(A, cluster, coeffs)
    
    f,ax = plt.subplots()
    plot_heat_map_from_data(A[0], Width, Height, args = (f,ax), title= f'Frame {ref_frame} coeffs map', cmap='inferno')  # inferno

#coeffs_map(file, ref_frame, thr_coeff,  search_q = (q_min, q_max))

files = cwd_files_search_with('.h5')
dropdown = ipywidgets.Dropdown(options= files, value = files[0], description='File : ', disabled=False)
frame = ipywidgets.IntSlider(value=0,                   min=0,           max= 15000, step=1, description='Frame : ', continuous_update=False, layout=ipywidgets.Layout(width='50%')) # reduce(lambda x,y: x*y-1 , width_height(dropdown.value) )
thr_coeff = ipywidgets.FloatSlider(value = 0.96 ,       min= 0,       max=1,    step=0.001 , description='THR Coeff:', continuous_update=False, layout=ipywidgets.Layout(width='50%'), readout=True, readout_format='.3f')    
search_q = ipywidgets.FloatRangeSlider( value=(1, 2), min=qgrid2[0],   max=qgrid2[-1], step=0.05,   description='q Range:',  disabled=False,  continuous_update=False,  orientation='horizontal',  readout=True, readout_format='.3f',layout=ipywidgets.Layout(width='50%') )
normalize = ipywidgets.Checkbox(value=True, description='Normalize', disabled=False, indent=False )
n_point = ipywidgets.IntSlider(value=4,                   min=1,         max=10, step=1, description='N-point : ', continuous_update=False, layout=ipywidgets.Layout(width='30%'))


ipywidgets.interactive(coeffs_map, file=dropdown, ref_frame=frame, thr_coeff=thr_coeff, search_q= search_q, normalize= normalize, L = n_point)

interactive(children=(Dropdown(description='File : ', options=('1934_B8.h5', '1934_B8_masked.h5', '1934_SN.h5'…

In [56]:
### checking convolution funcitons
L = 5
b = np.ones((1,L))/L
x = np.array([1,2,3,5,2,7]).reshape(1,-1)
print(signal.convolve(x,b, mode='same'))

L=5                        # L-point filter
b = (np.ones((1,L)))/L     # numerator co-effs of filter transfer function
a = np.ones(1)             # denominator co-effs of filter transfer function
y = signal.fftconvolve(x,b,mode='same',)   # filter output using convolution
print(y)
qgrid2[375]

[[1.2 2.2 2.6 3.8 3.4 2.8]]
[[1.2 2.2 2.6 3.8 3.4 2.8]]


1.6250000000000013

In [26]:
#### Mean shift unsupervised clustering algorithm ~ 3 mins to execute
from sklearn.cluster import MeanShift, estimate_bandwidth

# The following bandwidth can be automatically detected using
ms = MeanShift(n_jobs=-1)                 # sklearn mean-shift n_jobs = -1 means using all processors
ms.fit(X)                        # train mean shift algorithm
labels = ms.labels_              # get labels of trained data
plot_labels(file, labels)

number of estimated clusters : 15


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
%matplotlib widget

### import package
from sklearn.manifold import TSNE, MDS, LocallyLinearEmbedding
from functools import partial

### GMM model
def train_GMM(n_components):
    from sklearn.mixture import GaussianMixture               # import package
    gaussian_model = GaussianMixture(n_components=n_components)          # define the model
    gaussian_model.fit(X)                                     # train the model
    labels = gaussian_model.predict(X)                        # assign each data point to a cluster
    return labels

### t-SNE
def plot_tsne(X, labels, perplexity, n_iter):
    X_embedded = TSNE(n_components=3, perplexity=perplexity, n_iter=n_iter, init='pca', learning_rate=200, random_state=0, n_jobs=-1).fit_transform(X)
    #X_embedded = MDS(n_components=3, max_iter=100, n_init=1).fit_transform(X)
    
    # Set-up manifold methods
#     LLE = partial(LocallyLinearEmbedding, n_neighbors=10, n_components=3, eigen_solver='auto')
#     X_embedded = LLE(method='modified').fit_transform(X)
    
    tsne_data = np.vstack((X_embedded.T, labels)).T    # creating a new data frame which help us in ploting the result data
    #tsne_df = pd.DataFrame (data=tsne_data, columns= ("Dim_1", "Dim_2", "Dim_3", "label"))
    #sns.FacetGrid(tsne_df, hue="label", size=6).map(plt.scatter, 'Dim_1', 'Dim_2', 'Dim_3', 'label')
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.scatter(*zip(*tsne_data[:,:3]), c=tsne_data[:,3])
    plt.show()

labels = train_GMM(n_components=2)
plot_labels(file, labels, title='gaussian')
plot_tsne(X, labels, perplexity=50, n_iter=1000)

NameError: name 'X' is not defined

In [3]:
#### GMM and t-SNE plot
import functools

### figures output widgets
out1 = ipywidgets.Output()
out2 = ipywidgets.Output()

### GMM model
def train_GMM(X, n_components):
    from sklearn.mixture import GaussianMixture               # import package
    gaussian_model = GaussianMixture(n_components=n_components)          # define the model
    
    from sklearn import cluster
    #birch = cluster.Birch(n_clusters=n_components)
    
    gaussian_model.fit(X)                                     # train the model
    labels = gaussian_model.predict(X)                        # assign each data point to a cluster
    return labels

@out1.capture(clear_output=True)
def unsupervised_map(file, labels, clusters):
    f1, ax1 = plt.subplots(num=1)
    plot_labels(file, labels, title='GMM', args = (f1, ax1), cmap = discrete_cmap( N= len(np.unique(labels)), base_cmap = 'brg') )   # discrete_cmap( N= len(np.unique(labels)), base_cmap = 'brg') 'Pastel1'

@out2.capture(clear_output=True)
def manifold_plot(X, labels, perplexity, n_iter, animation):
    tsne_data = manifold_tsne(X, perplexity, n_iter, dim=3)
    print(animation)
    f2, ax2 = plt.subplots(num=2)
    plot_3d(tsne_data, labels, args = (f2,ax2), cmap = discrete_cmap( N= len(np.unique(labels)), base_cmap = 'brg'))  # discrete_cmap( N= len(np.unique(labels)), base_cmap = 'bwr')   'Pastel1'
    if animation: plot_3d_animation(tsne_data, labels, elev=35, azim=60, anim_frames=100, anim_interval=10)

def on_button_clicked_params():
    return dropdown.value, search_q.value, n_point.value, normalize.value, clusters.value, perplexity.value, n_iter.value, animation.value

def on_button_clicked(_):   # 
    %matplotlib widget
    
    ### get function arguments
    dropdown, search_q, n_point, normalize, clusters, perplexity, n_iter, animation = on_button_clicked_params()
    
    ### Data preprocessing
    q_min, q_max = search_q
    X = file_preprocess(file = dropdown, L=n_point, qgrid=qgrid2, q_min=q_min, q_max=q_max, normalize=normalize)
    
    ### Unsupervised model training and visualizing
    labels = train_GMM(X, n_components=clusters)
    labels = sort_labels(labels)
    unsupervised_map(dropdown, labels, clusters)
    manifold_plot(X, labels, perplexity, n_iter, animation)
    
### file selection and operation widgets  
files      = cwd_files_search_with('.h5')
dropdown   = ipywidgets.Dropdown(options= files, value = files[0], description='File : ', disabled=False)
search_q   = ipywidgets.FloatRangeSlider( value=(1.0, 2.0), min=qgrid2[0],   max=qgrid2[-1], step=0.0001,   description='q Range:',  disabled=False,  continuous_update=False,  layout=ipywidgets.Layout(width='40%'), orientation='horizontal',  readout=True, readout_format='.3f',)
n_point    = ipywidgets.IntSlider(value=4,                   min=1,         max=10, step=1, description='N-point : ', continuous_update=False, )
normalize  = ipywidgets.Checkbox(value=False, description='Normalize', disabled=False, indent=False )

### clustering and manifold widgets
clusters   = ipywidgets.IntSlider(value=4, min=1, max=20, step=1, description='clusters', continuous_update=False          )
perplexity = ipywidgets.IntSlider(value=30, min=10, max=1000, step=10, description='perplexity', continuous_update=False   )
n_iter     = ipywidgets.IntSlider(value=1000, min=500, max=10000, step=500, description='n_iter', continuous_update=False  )
button     = ipywidgets.Button(description='Go!');                                      button.style.button_color = "green";

button.on_click(on_button_clicked)

### animation object
animation  = ipywidgets.Checkbox(value=False, description='Animation', disabled=False, indent=False )


display(ipywidgets.HBox([dropdown, search_q, n_point, normalize, animation ]))
display(ipywidgets.HBox([ipywidgets.VBox([clusters, perplexity, n_iter, button]), out1, out2]))

In [8]:
import functools
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Click Me!")
display(button)



def fun(a):
    print(a)

def on_button_clicked(b, rs_="some_default_string"):
    fun(rs_)

button.on_click(functools.partial(on_button_clicked, rs_="abcdefg"))

Button(description='Click Me!', style=ButtonStyle())

In [18]:
### difference intensity heatmap 
out = ipywidgets.Output()


@out.capture(clear_output=True)
def on_button_clicked(_):   
    %matplotlib widget

    Width, Height = width_height(file=dropdown.value)
    
    ### Data preprocessing
    q_min, q_max = search_q.value
    X = file_preprocess(file = dropdown.value, L=n_point.value, qgrid=qgrid2, q_min=q_min, q_max=q_max, normalize=False)
    diff_patterns = find_rep_value(qgrid2, X, method = 'circ')
    filtered_diff_patterns = snaking(Width, Height, diff_patterns)
    filtered_diff_patterns = filtered_diff_patterns.reshape(Height,Width)

    ### original data
    Iq = read_Iq(file = dropdown.value, scattering = 'merged')
    q_min_idx, q_max_idx = qgrid_to_indices(qgrid2, q_min), qgrid_to_indices(qgrid2, q_max)
    Iq = Iq[:,q_min_idx:q_max_idx]
    diff_patterns = find_rep_value(qgrid2, Iq, method = 'circ')
    img_orig = snaking(Width, Height, diff_patterns)
    img_orig = img_orig.reshape(Height,Width)

    subtracted = (img_orig-filtered_diff_patterns).reshape(Height,Width)

    fig, [ax1, ax2, ax3] = plt.subplots(nrows=1, ncols=3, figsize=(16,3))
    
    plot_heat_map_from_data(img_orig, Width, Height, args = (fig, ax1), title= 'original', cmap="viridis")
    plot_heat_map_from_data(filtered_diff_patterns, Width, Height, args = (fig, ax2), title= 'filtered', cmap="viridis")
    plot_heat_map_from_data(subtracted, Width, Height, args = (fig, ax3), title= 'subtracted', cmap="viridis")
    
    
### file selection and operation widgets  
files      = cwd_files_search_with('.h5')
dropdown   = ipywidgets.Dropdown(options= files, value = files[0], description='File : ', disabled=False)
search_q   = ipywidgets.FloatRangeSlider( value=(1, 2), min=qgrid2[0],   max=qgrid2[-1], step=0.1,   description='q Range:',  disabled=False,  continuous_update=False,  orientation='horizontal',  readout=True, readout_format='.2f',)
n_point    = ipywidgets.IntSlider(value=4,                   min=1,         max=10, step=1, description='N-point : ', continuous_update=False, layout=ipywidgets.Layout(width='30%'))

#### button press event
button     = ipywidgets.Button(description='Go!');                                      button.style.button_color = "green";
button.on_click(on_button_clicked)

display(ipywidgets.HBox([dropdown, search_q, n_point, button ]))
display(out)

Output()

In [6]:
### 2-d plot animation
%matplotlib widget

from matplotlib.animation import FuncAnimation
from IPython import display

x_data = []
y_data = []

# Plot setups
fig, ax = plt.subplots()
plt.axis('scaled')   # making both x and y axis scaled
ax.set_xlim(0,20)
ax.set_ylim(0,200)
line, = ax.plot([])

# things that are going to change in every frame
def animate(frame):
    
    # append new data/update plot 
    x_data.append(frame)
    y_data.append(frame*10)
    
    # plot new data
#     line.set_xdata(x_data)
#     line.set_ydata(y_data)
    line.set_data((x_data,y_data))

    return line, # return plotting object for each frame call

anim = FuncAnimation(fig, func=animate, frames=np.arange(0,10,0.1), interval=100)   # smooth --> interval = 20, frames =100 big changes --> interval = 100
video = anim.to_html5_video()   # visualize in jupyter-lab
html = display.HTML(video)      # convert to html
display.display(html)           # display HTML file
plt.close()                     # close plot

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
### 3-d plot animation

%matplotlib widget
X = np.array([[1,1,1], [2,2,3],[10,9,8]])
labels = np.array([1,2,10])
plot_3d_animation(X, labels,)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …